In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import pysal as ps
%matplotlib inline
%config IPCompleter.greedy=True

/home/steve/anaconda3/lib/python3.7/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://migrating.pysal.org
  ), VisibleDeprecationWarning)


In [2]:
df = pd.read_csv('fent_county_year.txt', sep = '\t')
fent_all_years = pd.read_csv('all_fentanyl_by_county.txt', sep = '\t')
fent_13_17 = pd.read_csv('all_fent_by_county_13-17.txt', sep = '\t')
national_fent = pd.read_csv('national_fent.txt', sep = '\t')
national_all_opioids = pd.read_csv('grouped_by_year_not_county_all_opioids.txt', sep = '\t')
national_fent_cocaine = pd.read_csv('fent_and_cocaine.txt', sep = '\t')
fent_by_state = pd.read_csv('fent_state.txt', sep = '\t')

In [6]:
df = df.drop('Notes', axis = 'columns')
df = df.dropna()
df = df.drop('Year Code', axis = 'columns')
df.columns = ['county', 'county_code', 'year', 'deaths', 'population', 'crude_rate', 'aa_rate']

fent_all_years = fent_all_years.drop('Notes', axis = 'columns')
fent_all_years = fent_all_years.dropna()
fent_all_years.columns = ['county', 'county_code', 'deaths', 'population', 'crude_rate', 'aa_rate']

fent_13_17 = fent_13_17.drop('Notes', axis = 'columns')
fent_13_17 = fent_13_17.dropna()
fent_13_17.columns = ['county', 'county_code', 'deaths', 'population', 'crude_rate', 'aa_rate']

national_fent = national_fent.drop('Notes', axis = 'columns')
national_fent = national_fent.dropna()
national_fent = national_fent.drop('Year Code', axis = 'columns')
national_fent.columns = ['year', 'deaths', 'population', 'crude_rate', 'aa_rate']

national_all_opioids = national_all_opioids.drop('Notes', axis = 'columns')
national_all_opioids = national_all_opioids.dropna()
national_all_opioids = national_all_opioids.drop('Year Code', axis = 'columns')
national_all_opioids.columns = ['year', 'deaths', 'population', 'crude_rate']

national_fent_cocaine = national_fent_cocaine.drop('Notes', axis = 'columns')
national_fent_cocaine = national_fent_cocaine.dropna()
national_fent_cocaine = national_fent_cocaine.drop('Year Code', axis = 'columns')
national_fent_cocaine.columns = ['year', 'deaths', 'population', 'crude_rate', 'aa_rate']

fent_by_state = fent_by_state.drop(['Notes', 'Year Code'], axis = 'columns')
fent_by_state = fent_by_state.dropna()
fent_by_state.columns = ['state', 'state_code', 'year', 'deaths', 'population', 'crude_rate', 'aa_rate']

In [42]:
# dropping hawaii and alaska because they make the map ugly
df['state'] = df.county.apply(lambda x: x[-2:])
df = df[df['state'] != 'AK']
df = df[df['state'] != 'HI']
fent_13_17['state'] = fent_13_17['county'].apply(lambda x: x[-2:])
fent_13_17 = fent_13_17[fent_13_17['state'] != 'AK']
fent_13_17 = fent_13_17[fent_13_17['state'] != 'HI']
fent_by_state_13on = fent_by_state[fent_by_state.year > 2012]
fent_by_state_13on = fent_by_state_13on[fent_by_state_13on.state != 'Alaska']
fent_by_state_13on = fent_by_state_13on[fent_by_state_13on.state != 'Hawaii']

In [71]:
df = df.drop(df[df['deaths'] == 'Missing'].index)

/home/steve/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


TypeError: invalid type comparison

In [72]:
df = df.drop(df[df.county == 'Bedford city, VA'].index)

In [25]:
# for now put suppressed entries to 0
def convert_suppressed(entry):
    if entry == 'Suppressed':
        return 0
    elif entry == 'Missing':
        return 0
    else:
        return int(entry)
    
df['deaths'] = df['deaths'].apply(convert_suppressed)
fent_all_years['deaths'] = fent_all_years['deaths'].apply(convert_suppressed)
fent_13_17['deaths'] = fent_13_17['deaths'].apply(convert_suppressed)
national_fent['deaths'] = national_fent['deaths'].apply(convert_suppressed)
national_all_opioids['deaths'] = national_all_opioids['deaths'].apply(convert_suppressed)
national_fent_cocaine['deaths'] = national_fent_cocaine['deaths'].apply(convert_suppressed)
fent_by_state['deaths'] = fent_by_state['deaths'].apply(convert_suppressed)

In [35]:
print(sum(fent_13_17.deaths == 0))
sum(fent_13_17.deaths > 0)

2338


809

In [28]:
sum(fent_all_years.deaths == 0)

1971

In [18]:
len(fent_13_17)

3147

In [29]:
national_fent[national_fent.year > 2012].deaths.sum()

66108

In [30]:
sum(fent_13_17.deaths)

61262

In [45]:
has_2017 = list(df[(df.year == 2017) & (df.deaths > 0)].county)
has_2016 = list(df[(df.year == 2016) & (df.deaths > 0)].county)
has_2015 = list(df[(df.year == 2015) & (df.deaths > 0)].county)
has_2014 = list(df[(df.year == 2014) & (df.deaths > 0)].county)
has_2013 = list(df[(df.year == 2013) & (df.deaths > 0)].county)

missing_2017 = list(df[(df.year == 2017) & (df.deaths == 0)].county)
missing_2016 = list(df[(df.year == 2016) & (df.deaths == 0)].county)
missing_2015 = list(df[(df.year == 2015) & (df.deaths == 0)].county)
missing_2014 = list(df[(df.year == 2014) & (df.deaths == 0)].county)
missing_2013 = list(df[(df.year == 2013) & (df.deaths == 0)].county)

In [53]:
interpolate_16_17 = [x for x in missing_2016 if x in has_2017]

In [58]:
fent_16plus17 = pd.read_csv('fent_county_only1617.txt', sep = '\t')
fent_16plus17 = fent_16plus17.drop(['Notes'], axis = 'columns')
fent_16plus17.dropna(inplace = True)
fent_16plus17.columns = ['county', 'county_code', 'deaths', 'population', 'crude_rate']
fent_16plus17['deaths'] = fent_16plus17['deaths'].apply(convert_suppressed)


In [140]:
hdf = df[df.year == 2017][['county', 'county_code', 'state']]
hdf['deaths13'] = list(df[df.year == 2013].deaths)
hdf['deaths14'] = list(df[df.year == 2014].deaths)
hdf['deaths15'] = list(df[df.year == 2015].deaths)
hdf['deaths16'] = list(df[df.year == 2016].deaths)
hdf['deaths17'] = list(df[df.year == 2017].deaths)
hdf.reset_index(drop = True, inplace = True)

In [141]:
hdf['d1617'] = hdf.merge(fent_16plus17[['county', 'deaths']], on = 'county', how = 'left').deaths

In [142]:
hdf

,county,county_code,state,deaths13,deaths14,deaths15,deaths16,deaths17,d1617
0,"Autauga County, AL",1001.0,AL,0,0,0,0,0,0
1,"Baldwin County, AL",1003.0,AL,0,0,0,0,0,10
2,"Barbour County, AL",1005.0,AL,0,0,0,0,0,0
3,"Bibb County, AL",1007.0,AL,0,0,0,0,0,0
4,"Blount County, AL",1009.0,AL,0,0,0,0,0,0
5,"Bullock County, AL",1011.0,AL,0,0,0,0,0,0
6,"Butler County, AL",1013.0,AL,0,0,0,0,0,0
7,"Calhoun County, AL",1015.0,AL,0,0,0,0,0,0
8,"Chambers County, AL",1017.0,AL,0,0,0,0,0,0
9,"Cherokee County, AL",1019.0,AL,0,0,0,0,0,0


In [108]:
fent_16plus17

,county,county_code,deaths,population,crude_rate
0,"Autauga County, AL",1001.0,0,110920,Suppressed
1,"Baldwin County, AL",1003.0,10,421191,Unreliable
2,"Barbour County, AL",1005.0,0,51235,Suppressed
3,"Bibb County, AL",1007.0,0,45311,Suppressed
4,"Blount County, AL",1009.0,0,115717,Suppressed
5,"Bullock County, AL",1011.0,0,20671,Suppressed
6,"Butler County, AL",1013.0,0,39823,Suppressed
7,"Calhoun County, AL",1015.0,0,229339,Suppressed
8,"Chambers County, AL",1017.0,0,67556,Suppressed
9,"Cherokee County, AL",1019.0,0,51582,Suppressed


In [109]:
hdf.merge(fent_16plus17[['county', 'deaths']], on = 'county', how = 'left')

,county,county_code,state,deaths13,deaths14,deaths15,deaths16,deaths17,ideaths16,deaths
0,"Autauga County, AL",1001.0,AL,0,0,0,0,0,0.0,0
1,"Baldwin County, AL",1003.0,AL,0,0,0,0,0,0.0,10
2,"Barbour County, AL",1005.0,AL,0,0,0,0,0,0.0,0
3,"Bibb County, AL",1007.0,AL,0,0,0,0,0,0.0,0
4,"Blount County, AL",1009.0,AL,0,0,0,0,0,0.0,0
5,"Bullock County, AL",1011.0,AL,0,0,0,0,0,0.0,0
6,"Butler County, AL",1013.0,AL,0,0,0,0,0,0.0,0
7,"Calhoun County, AL",1015.0,AL,0,0,0,0,0,0.0,0
8,"Chambers County, AL",1017.0,AL,0,0,0,0,0,19.0,0
9,"Cherokee County, AL",1019.0,AL,0,0,0,0,0,0.0,0


In [132]:
hdf.merge(fent_16plus17[['county', 'deaths']], on = 'county', how = 'left').deaths - hdf['deaths17']

0        0
1       10
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       1
28       0
29       0
        ..
3078     0
3079    13
3080    10
3081     0
3082     0
3083     5
3084     0
3085     0
3086     0
3087     0
3088     0
3089     0
3090     0
3091     0
3092     0
3093     0
3094     0
3095     0
3096     0
3097     0
3098     0
3099     0
3100     0
3101     0
3102     0
3103     0
3104     0
3105     0
3106     0
3107     0
Length: 3108, dtype: int64

In [143]:
national_fent

,year,deaths,population,crude_rate,aa_rate
0,1999.0,730,279040168.0,0.3,0.3
1,2000.0,782,281421906.0,0.3,0.3
2,2001.0,957,284968955.0,0.3,0.3
3,2002.0,1295,287625193.0,0.5,0.4
4,2003.0,1400,290107933.0,0.5,0.5
5,2004.0,1664,292805298.0,0.6,0.6
6,2005.0,1742,295516599.0,0.6,0.6
7,2006.0,2707,298379912.0,0.9,0.9
8,2007.0,2213,301231207.0,0.7,0.7
9,2008.0,2306,304093966.0,0.8,0.8


In [145]:
national_fent[national_fent.year > 2012].deaths.sum()

66108

In [146]:
fent_13_17.deaths.sum()

61183